This puzzle was first proposed (1989) by [Gordon Lee](http://www.primepuzzles.net/puzzles/puzz_001.htm): given a grid of numbers, how many distinct primes can you find embedded in the matrix, regarding that you can read the lines or part of them, in form vertical, horizontal or diagonal orientation, in both directions.

Note that you can't change direction once you start moving (e.g. this isn't Boggle).

Input Description
You'll be given a single number on a line which tells you how many rows and columns to read (all grids will be square). Example:

    3 
    113
    754
    937

In [1]:
L = """3 
113
754
937,5 
11933
99563
89417
33731
32939,6
317333
995639
118142
136373
349199
379379,40
0251677085866837460317708637021446063144
8812262220360202463050064531874436437682
5251855367278508848642345043775871434078
0042675865438283025822603307175060748288
5672321632434878440388701468545837465571
3448326728143606881852187616524878044060
8876415778774852362710315274652021065556
1406474838287088561242126854006826771778
7827443331184330371521043472218803550383
6318874838447276075161123302780187880165
0884752758538865306583258450386821283658
1260362124615176735303563717773657467333
2580363145201308707655341168610513145546
4142635386876777348215436708254351251288
5301330463217260626047132625527161775404
8620446353006857360714856156584322276288
0813375760405334773480860674772272733638
6715558007108501053612008324501255710425
8840634327383685827335506853781648727036
8827728873376824454871536655067801443735
0664640563836487481174816586572628815173
7186752536147276768154002317573417465332
4438770023402783205544064640821537621225
4162442401558771474140203865162080237721
5008757506737224070577338578644664641338
2155803687408638660278862273674652462840
2118148017744113203720114756276821067158
4838003412436782114402742024145245437315
5161343527676283186170466281455700086618
7723886261287175705152273086317588317188
6653360024271146551000054710768617586846
0050014847531086708661266564560614115164
3351156208161708784441387827072734346251
0457546342466313073230326436563643506534
3837451141488371231210888733717540046582
3334248265835234158638343058444640886465
0173240266426385002380821305357684721128
0437020214873361352055818843664073456138
3858604586068245520287541000014334760058
5840781588142205318614583635575571714673""".split(',')


inx = """6
317333
995639
118142
136373
349199
379379"""

import numpy as np
from sympy import isprime

def prep_data(inx):
    r = inx.split('\n')
    A = np.array([[x for x in y.strip()] for y in r[1:]])
    s = int(r[0])
    S = set()
    
    dia = [A[::-1,:].diagonal(i) for i in range(-s+1,s)]
    dia += [A.diagonal(i) for i in range(s-1,-s,-1)]
    dia += [A[i] for i in range(s)]
    dia += [A.T[i] for i in range(s)]
    
    def substrings(L):
        L_rev = L[::-1]
        l = len(L)
        for x in range(l):
            n = l-x
            for y in range(n):
                S.add(int(L[x:x+y+1]))
                S.add(int(L_rev[x:x+y+1]))
    for x in dia:
        substrings(''.join(x.tolist()))
    return S

%timeit -n1 -r1 sum(isprime(x) for x in prep_data(inx) if x > 1)

1 loop, best of 1: 10.8 ms per loop


In [7]:
import numpy as np
import time
from random import randint

def grid_primes(s):
    start = time.time()

    size = int(s[:s.find('\n')])
    grid = np.array(list(s[2:].replace('\n', ''))).reshape([size, -1])
    lines, seq = set(), set()

    for i in range(size):
        lines.add(''.join(grid[:, i])) # columns
        lines.add(''.join(grid[i])) # rows

    for _ in range(4):
        for i in range(size):
            lines.add(''.join(grid.diagonal(i))) # diagonals
        grid = np.rot90(grid) 

    for line in lines:
        for i in range(len(line)):
            for j in range(i + 1, size + 1):
                if line[i:j][0] != '0':
                    seq.add(line[i:j])
                if line[i:j][::-1][0] != '0':
                    seq.add(line[i:j][::-1])

    def miller_rabin_test(n):
        k, t, d = 100, 0, n - 1

        if n == 1:
            return False
        if n == 2:
            return True
        if n % 2 == 0:
            return False

        while d % 2 == 0:
            t += 1
            d //= 2
        for _ in range(k):
            a = randint(2, n - 1)
            res = pow(a, d, n)
            if res == 1:
                continue
            for _ in range(t - 1):
                if res == n - 1:
                    break
                res = pow(res, 2, n)
            if res != n - 1:
                return False
        return True

    seq = [int(i) for i in seq]   
    primes = len([n for n in seq if miller_rabin_test(n)])

    print('{} ({} secs)'.format(primes, round(time.time() - start, 3)))

for x in L:
    grid_primes(x)

30 (0.031 secs)
116 (0.139 secs)
187 (0.215 secs)
5487 (18.048 secs)
